In [124]:
from twython import Twython
import pandas as pd
from tqdm import tqdm

In [4]:
python_tweets = Twython('KytN5DmtRxSTGeC4MmEDDRjF3', 'SarGEbusNDeD8ydSM290pz2FMeQMCEvnrRJ7Cur26EiWsCuiij')

In [264]:
query = {'q': 'украина',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'ru',
        'until': '2022-04-12'
        }

In [266]:
search_result = python_tweets.search(**query)['statuses']

In [267]:
tweets_dict = {'id':[], 'id_str':[], 'created_at':[], 'text':[], 
               'name':[], 'screen_name':[],
               'location':[], 'followers_count':[], 'geo':[],
               'place':[], 'retweet_count':[], 'favorite_count':[]}
for tweet in search_result:
    tweets_dict['id'].append(tweet['id'])
    tweets_dict['id_str'].append(tweet['id_str'])
    tweets_dict['created_at'].append(tweet['created_at'])
    tweets_dict['text'].append(tweet['text'])
    tweets_dict['name'].append(tweet['user']['name'])
    tweets_dict['screen_name'].append(tweet['user']['screen_name'])
    tweets_dict['location'].append(tweet['user']['location'])
    tweets_dict['followers_count'].append(tweet['user']['followers_count'])
    tweets_dict['geo'].append(tweet['geo'])
    tweets_dict['place'].append(tweet['place'])
    tweets_dict['retweet_count'].append(tweet['retweet_count'])
    tweets_dict['favorite_count'].append(tweet['favorite_count'])
    
df = pd.DataFrame(tweets_dict)

In [268]:
df

,id,id_str,created_at,text,name,screen_name,location,followers_count,geo,place,retweet_count,favorite_count


In [86]:
df['text'] = df['text'].str.upper()

In [87]:
df['text'].str.contains('ПУТИН').value_counts()

False    96
True      4
Name: text, dtype: int64

In [90]:
df['text'].str.contains('УКРАИН').value_counts()

True     81
False    19
Name: text, dtype: int64

In [97]:
df[~df['text'].str.contains('УКРАИН')]['text'][3]

'На Фиджи арестовали яхту, которую связывают с российским олигархом Керимовым - https://t.co/xKXJv69oaP #Киев… https://t.co/xylg6T1jEB'

# Collect tweets based on keywords

In [278]:
special_words = ['война', 'украина', 'россия', 'спецоперация', 'киев', 'мин обороны', 'луганск', 'донецк',
                 'лнр', 'днр', 'донбас', 'путин', 'нато', 'мариуполь', 'харьков', 'военные', 'мин обороны', 'санкции',
                 'зеленский']

In [279]:
dates = ['2022-04-13', '2022-04-14', '2022-04-15', '2022-04-16', 
         '2022-04-17', '2022-04-18', '2022-04-19', '2022-04-20']

In [286]:
len(special_words)*len(dates)

152

In [241]:
df = pd.DataFrame(columns=['id', 'id_str', 'created_at', 'text', 'name', 'screen_name', 'location',
       'followers_count', 'geo', 'place', 'retweet_count', 'favorite_count'])

In [291]:
for word in tqdm(special_words):
    for date in dates:
        if len(df) == 0:
            since_id_value = 0
        else:
            since_id_value = int(df['id'].max())

        query = {'q': word,
        'result_type': 'recent',
        'count': 100,
        'lang': 'ru',
        'until': date,
        #'since_id': since_id_value
        }

        search_result = python_tweets.search(**query)['statuses']

        tweets_dict = {'id':[], 'id_str':[], 'created_at':[], 'text':[], 
               'name':[], 'screen_name':[],
               'location':[], 'followers_count':[], 'geo':[],
               'place':[], 'retweet_count':[], 'favorite_count':[]}

        for tweet in search_result:
            try:
                place = tweet['place']['country']
            except:
                place = 'NoData'
            tweets_dict['id'].append(tweet['id'])
            tweets_dict['id_str'].append(tweet['id_str'])
            tweets_dict['created_at'].append(tweet['created_at'])
            tweets_dict['text'].append(tweet['text'])
            tweets_dict['name'].append(tweet['user']['name'])
            tweets_dict['screen_name'].append(tweet['user']['screen_name'])
            tweets_dict['location'].append(tweet['user']['location'])
            tweets_dict['followers_count'].append(tweet['user']['followers_count'])
            tweets_dict['geo'].append(tweet['geo'])
            tweets_dict['place'].append(place)
            tweets_dict['retweet_count'].append(tweet['retweet_count'])
            tweets_dict['favorite_count'].append(tweet['favorite_count'])

        df2 = pd.DataFrame(tweets_dict)

        df = pd.concat([df,df2])

    #df['id'] = df['id'].astype(int)

100%|███████████████████████████████████████████| 19/19 [02:03<00:00,  6.50s/it]


In [292]:
df = df.reset_index(drop='index')

df.shape

(116736, 12)

In [293]:
df.drop_duplicates().shape

(25798, 12)

In [295]:
df.drop_duplicates().to_csv('tweets.csv', index=False)

In [258]:
df[['text']].drop_duplicates()

,text
0,RT @mechtacariya: Вот видео резонансного заявл...
1,RT @afonja20142: Австрийский канцлер К. Нехамм...
2,RT @wargonzoo: ⚡️Как оркестр штурмовал Мариупо...
3,@svtv_news Братоубийственной война - чушь. Рус...
4,"RT @prof_preobr: Вы спрашивали, до каких глуби..."
...,...
17330,Владимир Зеленский передал главе представитель...
17331,Вторая фаза войны. Российские войска начали «б...
17332,Зеленский: российские войска начали наступлени...
17381,"RT @WKNtuUnoJQpnIMP: Зеленский визжит, что нач..."


In [218]:
df.sample(10)

,id,id_str,created_at,text,name,screen_name,location,followers_count,geo,place,retweet_count,favorite_count
1248,1515113585038569476,1515113585038569476,Fri Apr 15 23:43:22 +0000 2022,"Запрещённая Украина, [15 апр. 2022 в 23:00]\nК...",Vlad Khabitsov (Хабиц. Влад),vladneizvestnii,Алания,842,None,NoData,0,1
4070,1513647414275710981,1513647414275710981,Mon Apr 11 22:37:20 +0000 2022,"RT @dayanat01: Мариуполь наш, поступает информ...",Manana Kasoeva,KasoevaManana,,1,None,NoData,187,0
4863,1513942403497005067,1513942403497005067,Tue Apr 12 18:09:31 +0000 2022,RT @JoN_TM: Демилитаризация и денацификация\nИ...,Denys Kuznetsov,dnk_inc_ua,,1,None,NoData,101,0
5288,1515032549038383106,1515032549038383106,Fri Apr 15 18:21:22 +0000 2022,@maxidax @bbcrussian А ты знаешь где стоят сей...,ЯСЬКА 🇺🇦,ssnechiporik191,,231,None,NoData,0,2
1002,1514256534259257348,1514256534259257348,Wed Apr 13 14:57:45 +0000 2022,Россия и Украина продолжают переговоры в онлай...,РИА Новости,rianru,Russia,2672882,None,NoData,16,31
6720,1514742583322423300,1514742583322423300,Thu Apr 14 23:09:09 +0000 2022,❗️ Дислокация основных формирований и объектов...,НеМожетБыть,Ne_Mojet_Byt,,214,None,NoData,0,0
7544,1515115607993335813,1515115607993335813,Fri Apr 15 23:51:25 +0000 2022,Потери украинских военных в войне с Россией оц...,Дмитрий Гордон,dmitry_gordon,Украина,110146,None,NoData,15,91
2883,1515083436540796936,1515083436540796936,Fri Apr 15 21:43:34 +0000 2022,"RT @VictorKvert2008: Лавров: ""Потопление крейс...",Ray Gun,RayGun31328402,,48,None,NoData,505,0
6553,1514389860752060418,1514389860752060418,Wed Apr 13 23:47:33 +0000 2022,"RT @pan_dok: Я правильно понимаю, если фашистс...",Oleksandr Shvets🇺🇦,cidonix,Украина,137,None,NoData,39,0
1263,1515110247341432838,1515110247341432838,Fri Apr 15 23:30:07 +0000 2022,"Теперь понятно, что имели ввиду высокопоставле...",Председатель Ночи,21_yuriy,Русское Поволжье,1023,None,NoData,18,52
